In [1]:
import pandas as pd
import numpy as np


In [2]:
train = pd.read_csv('Train.csv')
test = pd.read_csv('Test.csv')

In [7]:
train.head(10)

,Image_id,Label
0,id_02amazy34fgh2.jpg,1
1,id_02mh3w48pmyc9.jpg,0
2,id_02rpb463h9d3w.jpg,0
3,id_02wc3jeeao8ol.jpg,1
4,id_03t2hapb8wz8p.jpg,1
5,id_04xrxyd43rlgz.jpg,0
6,id_05gqx7h8y6iye.jpg,1
7,id_06le9g89kqth5.jpg,1
8,id_082w0qygo3eth.jpg,0
9,id_086yvmu3nnvbe.jpg,1


In [8]:
test.head(10)

,Image_id
0,id_00exusbkgzw1b.jpg
1,id_03dqinf6w0znv.jpg
2,id_046yl0cxn3ybz.jpg
3,id_04athdtx2abyg.jpg
4,id_062aauf9e9jk0.jpg
5,id_0695b3b665yet.jpg
6,id_06cueqs5yhekm.jpg
7,id_0826dbdkr8sxr.jpg
8,id_09re5drh85tbo.jpg
9,id_0adud1myz8tvz.jpg


In [4]:
import os
image_path =os.getcwd()+'/Images/'

In [5]:
print(len(os.listdir(image_path)))

2699


In [6]:
train.shape, test.shape

((1619, 2), (1080, 1))

In [9]:
import os
import random
import shutil
import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout
from keras.applications.vgg16 import VGG16
from keras.optimizers import Adam

2023-03-19 15:39:07.800438: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-19 15:39:08.300175: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-19 15:39:08.300325: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-19 15:39:12.301568: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [10]:
data_dir = os.getcwd()+'/Images/'
train_dir = os.getcwd()+'/train/'
test_dir = os.getcwd()+'/test/'

In [19]:
import os
import shutil

# Create train and test directories
#train_dir = "train"
#test_dir = "test"
os.makedirs(train_dir, exist_ok=True)
os.makedirs(os.path.join(train_dir, "0"), exist_ok=True)
os.makedirs(os.path.join(train_dir, "1"), exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Split the dataset into train and test
for index, row in train.iterrows():
    src = os.path.join(data_dir, row["Image_id"])
    dst = os.path.join(train_dir, str(row["Label"]), row["Image_id"])
    shutil.copy(src, dst)
    
for index, row in test.iterrows():
    src = os.path.join(data_dir, row["Image_id"])
    dst = os.path.join(test_dir, row["Image_id"])
    shutil.copy(src, dst)


In [20]:
print(len(os.listdir(train_dir)))
print(len(os.listdir(test_dir)))

2
1080


In [21]:

# Set up the data generators
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode="binary"
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode="binary"
)


Found 1619 images belonging to 2 classes.
Found 0 images belonging to 0 classes.


In [23]:
# Set up the model
vgg16 = VGG16(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
for layer in vgg16.layers:
    layer.trainable = False

model = Sequential()
model.add(vgg16)
model.add(Flatten())
model.add(Dense(256, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(1, activation="sigmoid"))

model.compile(
    loss="binary_crossentropy",
    optimizer=Adam(lr=1e-4),
    metrics=["accuracy"]
)

2023-03-19 15:52:14.748885: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-03-19 15:52:14.748957: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-03-19 15:52:14.749024: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (temu): /proc/driver/nvidia/version does not exist
2023-03-19 15:52:14.750026: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


58889256/58889256 [==============================] - 8s 0us/step


/home/tevin/.local/lib/python3.10/site-packages/keras/optimizers/optimizer_v2/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [24]:

# Train the model
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=10,
    validation_data=test_generator,
    validation_steps=test_generator.samples // test_generator.batch_size
)

Epoch 1/10


2023-03-19 15:52:43.584421: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 411041792 exceeds 10% of free system memory.
2023-03-19 15:52:44.053763: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 411041792 exceeds 10% of free system memory.
2023-03-19 15:52:46.813877: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 205520896 exceeds 10% of free system memory.
2023-03-19 15:52:47.815779: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 205520896 exceeds 10% of free system memory.


 1/50 [..............................] - ETA: 16:48 - loss: 0.7644 - accuracy: 0.5000

2023-03-19 15:52:59.218255: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 411041792 exceeds 10% of free system memory.


50/50 [==============================] - 931s 19s/step - loss: 0.3953 - accuracy: 0.8144
Epoch 2/10
50/50 [==============================] - 863s 17s/step - loss: 0.1854 - accuracy: 0.9231
Epoch 3/10
50/50 [==============================] - 938s 19s/step - loss: 0.1318 - accuracy: 0.9590
Epoch 4/10
50/50 [==============================] - 941s 19s/step - loss: 0.1159 - accuracy: 0.9590
Epoch 5/10
50/50 [==============================] - 958s 19s/step - loss: 0.0749 - accuracy: 0.9742
Epoch 6/10
50/50 [==============================] - 987s 20s/step - loss: 0.0702 - accuracy: 0.9779
Epoch 7/10
50/50 [==============================] - 786s 16s/step - loss: 0.0731 - accuracy: 0.9716
Epoch 8/10
50/50 [==============================] - 751s 15s/step - loss: 0.0613 - accuracy: 0.9773
Epoch 9/10
50/50 [==============================] - 742s 15s/step - loss: 0.0587 - accuracy: 0.9824
Epoch 10/10
50/50 [==============================] - 758s 15s/step - loss: 0.0547 - accuracy: 0.9805


In [31]:
test

,Image_id
0,id_00exusbkgzw1b.jpg
1,id_03dqinf6w0znv.jpg
2,id_046yl0cxn3ybz.jpg
3,id_04athdtx2abyg.jpg
4,id_062aauf9e9jk0.jpg
...,...
1075,id_zv5fvjnakvf1r.jpg
1076,id_zvpikh1z30arn.jpg
1077,id_zypilwkudljyz.jpg
1078,id_zz9lwehh5sxdp.jpg


In [32]:
test_generator

In [34]:
import tensorflow as tf

In [35]:
# Convert the model to a TensorFlow Lite format
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Apply post-training quantization
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quant_model = converter.convert()

# Save the quantized model to a file
with open('model_quant.tflite', 'wb') as f:
    f.write(tflite_quant_model)

INFO:tensorflow:Assets written to: /tmp/tmpji3a1mub/assets


INFO:tensorflow:Assets written to: /tmp/tmpji3a1mub/assets
2023-03-19 18:43:50.531213: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2023-03-19 18:43:50.531291: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2023-03-19 18:43:50.534409: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmpji3a1mub
2023-03-19 18:43:50.551297: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2023-03-19 18:43:50.551412: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /tmp/tmpji3a1mub
2023-03-19 18:43:50.614768: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:357] MLIR V1 optimization pass is not enabled
2023-03-19 18:43:50.628675: I tensorflow/cc/saved_model/loader.cc:229] Restoring SavedModel bundle.
2023-03-19 18:43:51.175193: I tensorflow/cc/saved_model/loader.cc:213] Running initializatio

INFO:tensorflow:Assets written to: /tmp/tmpdq1mw75w/assets


INFO:tensorflow:Assets written to: /tmp/tmpdq1mw75w/assets
2023-03-19 18:44:14.555445: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2023-03-19 18:44:14.555535: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2023-03-19 18:44:14.556241: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmpdq1mw75w
2023-03-19 18:44:14.581107: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2023-03-19 18:44:14.581255: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /tmp/tmpdq1mw75w
2023-03-19 18:44:14.648908: I tensorflow/cc/saved_model/loader.cc:229] Restoring SavedModel bundle.
2023-03-19 18:44:15.091499: I tensorflow/cc/saved_model/loader.cc:213] Running initialization op on SavedModel bundle at path: /tmp/tmpdq1mw75w
2023-03-19 18:44:15.177618: I tensorflow/cc/saved_model/loader.cc:305] SavedModel